# Advanced Lane Lines Finding #


- **Calibrate** the camera by reading in chessboard images and locating their corners, to calculate the distortion coefficients
- Use the distortion coefs to undistort any new images before further processing
- Apply thresholding/gradients to get masked image of only the interesting pixels
- **Warp** the images of the road from the car camera to get the bird's eye view of the road
- Downsize the image 10x for the next step to work in real time
- **Use DBSCAN** to search for lane lines' pixels by exploiting adjacency and add to collection
- **Plot the lines using polynomial fit**
- **Scale the poly-line** and apply to the original size warped image
- **Overlap n subsequent images** to get better continuity and perform lane line detection 
- Record line and car position data
- Draw on road and inverse the warp taken before to get highlighted road markings and curvature

### Imports ###

In [1]:
import numpy as np
from statistics import mean
import cv2
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import pickle
from sklearn.cluster import DBSCAN
from moviepy.editor import VideoFileClip
from IPython.display import HTML
%matplotlib inline

### Parameter Board ###

In [2]:
# Thresholds for colors and gradients
thresh_x_grad = (35,255)
thresh_S = (90,255)
thresh_L = (90,255)

# Image dimensions
shape_x = 1280
shape_y = 720

# Scale factor
fx = 0.10
fy = 0.10

# DBSCAN parameters
epsilon = 25
min_samples = 30
invalid_centroids = 0

# Perspective transform matrix and inverse matrix
src = np.float32([(585,455), (695,455), (1100,720), (180,720)])
dst = np.float32([(240,0), (1040,0), (1040,720), (240,720)])
M = cv2.getPerspectiveTransform(src, dst)
Minv = cv2.getPerspectiveTransform(dst,src)


# load distortion coefficients
dist_pickle = pickle.load(open('dist_matrix.p', 'rb'))
distortion = dist_pickle['dist']
camera_mtx = dist_pickle['mtx']



### Lines Class ###

In [3]:
class Line():
    def __init__(self):
        self.detected = False
        self.last_fit = [np.array([False])]
        self.all_fit = []
        self.all_fit_ret = []
        self.last_n_fit = []
        self.avg_fit = None
        self.radius_of_curvature = None
        self.all_radii = []
        self.last_n_r = []
        self.last_n_x = []
        self.avg_x = []
        self.n = 20
        self.r_n = 20
        self.last_fit_weight = 7
        self.accepted_deviation = 0.55
        self.r_diff = 2000
        self.all_r_ret = []
        self.straight_line_radius = 10000
        self.fit_diff = []
        self.mx = 3.7/790
        self.my = 30/720
    
    def set_fit(self, fit):
        self.last_fit = fit
        self.all_fit.append(fit)
        if len(self.last_n_fit) < self.n:
            self.last_n_fit.append(fit)
        else:
            self.last_n_fit[:-1] = self.last_n_fit[1:]
            self.last_n_fit[-1] = fit
    
    def set_x(self, x):
        if len(self.last_n_x) < self.n :
            self.last_n_x.append(x)
        else:
            self.last_n_x[:-1] = self.last_n_x[1:]
            self.last_n_x[-1] = x
            
    def eval_avg(self):
        last_n_fit = np.array(self.last_n_fit)
        last_n_x = np.array(self.last_n_x)
        
        weights = np.ones(last_n_fit.shape[0])
        weights[-1] = self.last_fit_weight
        
        self.avg_fit = np.average(last_n_fit, axis=0, weights=weights)
        self.avg_x = np.average(last_n_x, axis=0, weights=weights)
    
    def get_radius(self, fit, y=719, append=True):
        mx= self.mx
        my=self.my
        
        a = fit[0]
        b = fit[1] 
        y = my*y
        
        a = mx * a /(my**2)
        b = mx * b / (my)
            
        r = ((1+(2*a*y+b)**2)**1.5 ) / np.absolute(2*a) 
        self.radius_of_curvature = r
        if append: 
            self.all_radii.append(r)
    
    def set_radius(self, r):
        if len(self.last_n_r) < self.r_n :
            self.last_n_r.append(r)
        else:
            self.last_n_r[:-1] = self.last_n_r[1:]
            self.last_n_r[-1] = r    
    
    def validate(self, fit, x):
        if self.avg_fit == None:
            self.set_fit(fit)
            self.set_x(x)
            self.eval_avg()
            self.all_fit_ret.append(self.avg_fit)
            self.get_radius(self.avg_fit)
            self.set_radius(self.radius_of_curvature)
            self.all_r_ret.append(self.radius_of_curvature)
            return self.avg_fit, self.avg_x
        
#        fit_diff = (np.abs((self.avg_fit-fit)/(self.avg_fit)))
#        self.fit_diff.append(fit_diff)
        self.get_radius(fit)
        if self.radius_of_curvature > self.straight_line_radius:
            self.set_fit(fit)
            self.set_x(x)
            self.eval_avg()
            #self.radius_of_curvature = 0
            self.all_fit.append(fit)
            self.all_fit_ret.append(self.avg_fit)
            self.all_r_ret.append(self.radius_of_curvature)
            return self.avg_fit, self.avg_x
        
        elif np.absolute(self.radius_of_curvature - (self.last_n_r[-1])) > self.r_diff:
            self.radius_of_curvature = np.average(self.last_n_r)
            self.set_radius(self.radius_of_curvature)
            self.all_fit.append(fit)
            self.all_fit_ret.append(self.avg_fit)
            self.all_r_ret.append(self.radius_of_curvature)
            return self.avg_fit, self.avg_x
        
        else:    
            self.set_fit(fit)
            self.set_x(x)
            self.eval_avg()
            self.all_fit_ret.append(self.avg_fit)
            #self.get_radius(self.avg_fit, append=False)
            self.set_radius(self.radius_of_curvature)
            self.radius_of_curvature = np.average(self.last_n_r)
            self.all_r_ret.append(self.radius_of_curvature)
            return self.avg_fit, self.avg_x
        

### Image overlap class ###
(Keeping track of last n images to overlap and get better fit)

In [4]:
class ImgOverlap():
    def __init__(self):
        self.n_frames = 2
        self.last_n_img = []
        
    def overlap(self, img):
        if len(self.last_n_img) == 0:
            self.last_n_img.append(img)
            return img
        elif len(self.last_n_img) < self.n_frames :
            self.last_n_img.append(img)
        else:
            self.last_n_img[:-1] = self.last_n_img[1:]
            self.last_n_img[-1] = img
        
        ovl = np.zeros_like(img)
        for i in range(len(self.last_n_img)):
            ovl = ovl + self.last_n_img[i]
        ovl[(ovl > 0)] = 1
        return ovl

### Global Variables ###

In [5]:
left_coefs = np.array([[0,0,0]], dtype=np.float32)
right_coefs = np.array([[0,0,0]], dtype=np.float32)

left_line = Line()
right_line = Line()
img_overlap = ImgOverlap()

### Utility Functions ###

In [6]:
def abs_sobel_thresh(img, orient='x', sobel_kernel=9, thresh=(35, 255)):
    #HLS = cv2.cvtColor(img, cv2.COLOR_RGB2HLS)
    HLS=img
    gray = HLS[:,:,0] # get R component
    sobel = cv2.Sobel(gray, cv2.CV_64F, int(orient=='x'), int(orient=='y'), ksize=sobel_kernel)
    sobel_abs = np.absolute(sobel)
    sobel_scaled = 255 * sobel_abs/np.max(sobel_abs)
    
    grad_binary = np.zeros_like(sobel_scaled)
    grad_binary[(sobel_scaled>thresh[0]) & (sobel_scaled<thresh[1])] = 1
    return grad_binary

def mag_thresh(img, sobel_kernel=9, thresh=(0, 255)):
    HLS = cv2.cvtColor(img, cv2.COLOR_RGB2HLS)
    gray = HLS[:,:,2] # get S component
    sobelx = cv2.Sobel(gray, cv2.CV_64F, 1, 0, ksize=sobel_kernel)
    sobely = cv2.Sobel(gray, cv2.CV_64F, 0, 1, ksize=sobel_kernel)
    
    sobel_mag = (sobelx**2 + sobely**2)**0.5
    sobel_scaled = 255 * sobel_mag/np.max(sobel_mag)
    
    mag_binary = np.zeros_like(sobel_scaled)
    mag_binary[(sobel_scaled>thresh[0]) & (sobel_scaled<thresh[1])] = 1
    return mag_binary

def dir_threshold(img, sobel_kernel=9, thresh=(0, np.pi/2)):
    HLS = cv2.cvtColor(img, cv2.COLOR_RGB2HLS)
    gray = HLS[:,:,2] # get S component
    sobelx = cv2.Sobel(gray, cv2.CV_64F, 1, 0, ksize=sobel_kernel)
    sobely = cv2.Sobel(gray, cv2.CV_64F, 0, 1, ksize=sobel_kernel)
    
    sobelx_abs = np.absolute(sobelx)
    sobely_abs = np.absolute(sobely)
    
    sobel_dir = np.arctan2(sobely_abs, sobelx_abs)
    
    dir_binary = np.zeros_like(sobel_dir)
    dir_binary[(sobel_dir>thresh[0]) & (sobel_dir<thresh[1])] = 1
    return dir_binary

def S_threshold(img, thresh=(90,255)):
    hls = cv2.cvtColor(img, cv2.COLOR_RGB2HLS)
    S = hls[:,:,2]
    binary = np.zeros_like(S)
    binary[(S>thresh[0]) & (S<=thresh[1])] = 1
    return binary

def L_threshold(img, thresh=(90,255)):
    hls = cv2.cvtColor(img, cv2.COLOR_RGB2HLS)
    L = hls[:,:,1]
    binary = np.zeros_like(L)
    binary[(L>thresh[0]) & (L<=thresh[1])] = 1
    return binary

def extract_thresh(img):
    S = S_threshold(img, thresh=thresh_S)
    L = L_threshold(img, thresh=thresh_L)
    X = abs_sobel_thresh(img, thresh=thresh_x_grad)
    
    binary = np.zeros_like(S)
    binary[(S>0) & (L>0) | (X>0)] = 1
    
    return binary

def apply_opening(img):
    kernel = np.ones((1,3), np.uint8)
    return cv2.morphologyEx(img, cv2.MORPH_OPEN, kernel)

def undistort(img):
    return cv2.undistort(img, camera_mtx, distortion, None, camera_mtx)

def warp_image(img):
    return cv2.warpPerspective(img, M, (shape_x, shape_y))

def unwarp_image(img):
    return cv2.warpPerspective(img, Minv, (shape_x, shape_y))

def rescale(img, fx=fx, fy=fy):
    return cv2.resize(img, (0,0), fx=0.1, fy=0.1)

def top_k_labels(y, k=2):
    uniq, count = np.unique(y, return_counts=True)
    if np.argwhere(uniq==-1).size!=0:
        count[np.argwhere(uniq==-1)]=0
    seq = np.argsort(count)
    top_k = []
    try:
        for i in range(k):
            top_k.append(uniq[seq[-1*(i+1)]])
        return np.array(top_k)
    except (IndexError):
        global invalid_centroids
        invalid_centroids += 1
        return None

def get_pointsets(img):
    dbscan = DBSCAN(eps=epsilon, min_samples=min_samples)
    nonzeros = np.array(np.nonzero(img), dtype=np.float32).T
    
    y = dbscan.fit_predict(nonzeros)
    
    top_labels = top_k_labels(y)
    
    if top_labels == None:
        return None
    else:
        return (nonzeros[y==top_labels[0]], nonzeros[y==top_labels[1]])

def fit_lines(ploty, fit1, fit2, fx=0.1, fy=0.1):
    left_fitx = (fit1[0]*(ploty*fy)**2 + fit1[1]*ploty*fy + fit1[2])/fx
    right_fitx = (fit2[0]*(ploty*fy)**2 + fit2[1]*ploty*fy + fit2[2])/fx
    
    # Seperate left and right point sets
    if np.average(left_fitx) < np.average(right_fitx):
        return left_fitx, right_fitx
    else:
        return right_fitx, left_fitx

def record_coefs(fit1, fit2, x1, x2):
    if fit1[2] < fit2[2]:
        fit_left = fit1
        fit_right = fit2
        left_fitx = x1
        right_fitx = x2
    else:
        fit_left = fit2
        fit_right = fit1
        left_fitx = x2
        right_fitx = x1
    
    global left_line, right_line
    fit_left, left_fitx = left_line.validate(fit_left, left_fitx)
    fit_right, right_fitx = right_line.validate(fit_right, right_fitx)
    
    return fit_left, fit_right, left_fitx, right_fitx

def draw_unwarp(img, warped, ploty, left_fitx, right_fitx, warped_ovl=None):
    # Create an image to draw the lines on - FROM Udacity class
    warp_zero = np.zeros_like(warped).astype(np.uint8)
    color_warp = np.dstack((warp_zero, warp_zero, warp_zero))

    # Recast the x and y points into usable format for cv2.fillPoly()
    pts_left = np.array([np.transpose(np.vstack([left_fitx, ploty]))])
    pts_right = np.array([np.flipud(np.transpose(np.vstack([right_fitx, ploty])))])
    pts = np.hstack((pts_left, pts_right))

    # Draw the lane onto the warped blank image
    cv2.fillPoly(color_warp, np.int_([pts]), (0,255, 0))

    # Warp the blank back to original image space using inverse perspective matrix (Minv)
    newwarp = cv2.warpPerspective(color_warp, Minv, (shape_x, shape_y)) 
    # Combine the result with the original image
    result = cv2.addWeighted(img, 1, newwarp, 0.3, 0)
    
    # Add the mini warped images as PIP on top-right of the returned image
    if warped_ovl !=None :
        pip = rescale(warped_ovl) * 255
        pip = np.dstack((pip, pip, pip))
        result[70:142, 1000:1128] = pip
    
    # Add text
    text = "radius = {:.3f}m".format(left_line.radius_of_curvature)
    text2 = "offset = {:.3f}m".format(get_offset(left_line, right_line))
    result = cv2.putText(result, text, (10,60), cv2.FONT_HERSHEY_SIMPLEX, 1,(255,255,255),2,cv2.LINE_AA)
    result = cv2.putText(result, text2, (800,60), cv2.FONT_HERSHEY_SIMPLEX, 1,(255,255,255),2,cv2.LINE_AA)
    
    return result

def get_offset(leftline, rightline):
    lane_width = np.min(rightline.avg_x) - np.min(leftline.avg_x)
    offset = (shape_x/2 - np.min(leftline.avg_x)) - lane_width/2
    
    #in metres
    offset = offset * leftline.mx
    return offset


### Image processing pipeline ###

In [7]:
def process_img(img):
    original = np.copy(img)
    img = undistort(img)
    img = extract_thresh(img)
    
    img = warp_image(img)
    img = apply_opening(img)
    
    global img_overlap
    img_ovl = img_overlap.overlap(img)
    img_scaled = rescale(img_ovl)
    
    clusters = get_pointsets(img_scaled)
    
    if clusters == None:
        fit1 = left_line.avg_fit
        fit2 = right_line.avg_fit
        # Get XY points
        ploty = np.linspace(0, shape_y-1, shape_y )
        left_fitx, right_fitx = fit_lines(ploty, fit1, fit2, fx=1, fy=1)
    else:
        fit1 = np.polyfit(clusters[0][:,0], clusters[0][:,1], 2) * [fy**2/fx, fy/fx, fx**-1 ]
        fit2 = np.polyfit(clusters[1][:,0], clusters[1][:,1], 2) * [fy**2/fx, fy/fx, fx**-1 ]
        
        # Get XY points
        ploty = np.linspace(0, shape_y-1, shape_y )
        left_fitx, right_fitx = fit_lines(ploty, fit1, fit2, fx=1, fy=1)
    
    # Store coefs to plot later
    fit1, fit2, left_fitx, right_fitx = record_coefs(fit1, fit2, left_fitx, right_fitx) 
    
    result = draw_unwarp(original, img, ploty, left_fitx, right_fitx, warped_ovl=img_ovl)
    return result

### Apply pipeline to Project Video (main) ###

In [8]:
video1 = 'project_video.mp4'
video1_output = 'project_video_output.mp4'
clip1 = VideoFileClip(video1)#.subclip(37,50)
imgs = clip1.fl_image(process_img)
%time imgs.write_videofile(video1_output, audio=False)

[MoviePy] >>>> Building video project_video_output.mp4
[MoviePy] Writing video project_video_output.mp4


100%|█████████████████████████████████████████████████████████████████████████████▉| 1260/1261 [02:47<00:00,  7.71it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: project_video_output.mp4 

Wall time: 2min 48s
